# Salt Write up
1. reading the protocol
```
# client message
data = '#'.join(data)
req = my_sk.public_key.encode() + bytes([command]) + box.encrypt(data.encode('utf8'), nonce)
```
1. identifying the vulnerabilities
 - nonce can be repeated

```
## client nonce creation
nonce = nacl.utils.random(Box.NONCE_SIZE)
nonce = bytes([nonce[0] & 0xFE]) + nonce[1:] # see comment in server

## server check of nonce
# the client already picked a random nonce for the request, so we can just increment it by one for the reply
if (nonce[0] & 1) == 1:
  raise Exception('bad nonce')
# 
answer_nonce = bytes([nonce[0] + 1]) + nonce[1:]
```

 - command is not encrypted
 - server has some interesting replies

```
if cmd == CMD_STORE:
    if len(params) < 2:
      return reply("you need to send more parameters! seriously, why would you just send one parameter? I should just let this fail with a silent error. sending just one parameter is stupid.")
```

1. how does the NaCl library works
 - ECDH
 - Stream cipher
 - MAC
1. making of a replay attack
1. extra: how to secure this
 - nonce proper
 - command autenticated

In [1]:
# intercepted request
request1 = b'\xa0f?u\xfb;AZf\xfc@{M!\xcdP\x92\xf6\x0f\xea\x1d\xad@\xc5\x8c\xd0R\xd8\xfdX81\x01d\xc8\x9b\xc4\xfd\x04\x9b\x843a\x940U\xc4\x7fa\x11W\xa9uf\xa9\xf4%w;`s[\xad\xa8V\x90\xe0w,\xb6<\xbd\xb1\xcbh=\x0b\x80\xba\xd8\x9bM\x17\xc6\x1f\x83<G\xcfV\x93\x00E\xe97\xcc\x9a.\xa1\xe6\x13\x11\xe9<\xae'
# intercepted response
response1 = b"e\xc8\x9b\xc4\xfd\x04\x9b\x843a\x940U\xc4\x7fa\x11W\xa9uf\xa9\xf4%\r6C\x8d\xe1Z\x95\xb1^\x92\xddF\xa7\xbb\x86\x19\xbaCW\xde\x9bo\xd3Z\x8d\x85kx\x81a\xb0\x0b\xc9\x14'L\xc6i\xc4V\x86=\xba\x11~\xcc\x9bw#i\xc7\xb2\xc0Z\x9d\x1d\xb3\x96\\\xf9\xffG\x8a\xa2"

# modified request, modified command and keeping public key, nonce and ciphertext
request2 = b'\xa0f?u\xfb;AZf\xfc@{M!\xcdP\x92\xf6\x0f\xea\x1d\xad@\xc5\x8c\xd0R\xd8\xfdX81\x00d\xc8\x9b\xc4\xfd\x04\x9b\x843a\x940U\xc4\x7fa\x11W\xa9uf\xa9\xf4%w;`s[\xad\xa8V\x90\xe0w,\xb6<\xbd\xb1\xcbh=\x0b\x80\xba\xd8\x9bM\x17\xc6\x1f\x83<G\xcfV\x93\x00E\xe97\xcc\x9a.\xa1\xe6\x13\x11\xe9<\xae'
# reply from server
response2 = b"e\xc8\x9b\xc4\xfd\x04\x9b\x843a\x940U\xc4\x7fa\x11W\xa9uf\xa9\xf4%\xcco\x95\xb8i\x10`Tz\xd6`\x8bJ&<i\xa5@C\x99\x8ed\xd9a\xce\x83}(\x86k\x81\x06\x9a&!P\xd5 \xdaT\x9d>\xbb\x11U\xc8\x8eek'\xc7\xa3\xed@\x86\x1d\xb0\x93A\xbc\xabI\x9f\xd1\xbcp\xf8\x1a\xba2u\xf8\x88\x05H\x16\xfe\xafX8K)\x03\xbb'&\x01\xcc\xaf\x1f\x1a{\x9dB\xb1\xb7\xc7\x17\x9ahe\x80'\x13\xdbG`\xf8~L\xcf@\x04Q\x97\xac\xf5\x157\x83b\xd4\x086\x08\xb1\xeb7\xf8*\xcf\x8a]q\xb1=\xc8\xd2\xb6\xc8\xdc\xf2)\x8e\x13\xd2\x16`\x84&Wn^\xce\xa8\x84\x9e\xc0\xeb\xb2\xc2\xcc\x92\xaf\x10\xad$c\x13\x92\xae]\x87\x83\x0c\x8a.D\xdf\xe0\xbf\x12\x87 \x11:"

knownplaintext = b"you need to send more parameters! seriously, why would you just send one parameter? I should just let this fail with a silent error. sending just one parameter is stupid."

# ciphertext we want to decrypt
c1 = response1[40:]
# ciphertext of known plaintext
c2 = response2[40:]

c3 = [c1[i]^c2[i]^knownplaintext[i] for i in range(len(c1))]

print("".join([chr(i) for i in c3]))

flag{no_crypto_is_invincible_against_stupidity}

